# Exploration 4
## Project: Make a nice lyric writer!
### 5 Steps in the project
1. 데이터 다운로드
2. 데이터 읽어오기
3. 데이터 정제
- Standardize
- Tokenize
- Vectorization
4. 평가 데이터셋 분리
5. 모델로 학습하기
6. 프로젝트 정리

### 참고
- [glob.glob(pathname, *, recursive=False)](https://docs.python.org/ko/3/library/glob.html) : 경로 지정을 포함하는 문자열인 pathname에 일치하는 경로 이름의 비어있을 수 있는 리스트를 반환합니다.
- splitlines(): 줄바꿈 기호에 따라 하나의 리스트로 저장해주는 메소드 


# 1. 데이터 다운로드

- mkdir -p ~/aiffel/lyricist/models
- ln -s ~/data ~/aiffel/lyricist/data

# 2. 데이터 읽어오기
- 사용할 라이브러리불러오기
- 파일 확인

In [1]:
import matplotlib.pyplot as plt
import os
import re
import glob
import shutil
import string

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples: \n", raw_corpus[:5])

데이터 크기: 187088
Examples: 
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights', "Just thinking how you've done me wrong"]


- 오늘의 Dataset은 Lyrics, 노래 가사입니다.
- TMI👀: Examples로 나온 가사는 Gloria Gaynor의 '[I will survive](https://youtu.be/ARt9HV9T0w8)'네요. 궁금하면 제목을 클릭해서 들어보세요! 듣자마자 다들 "아! 이 노래~😆" 하실 겁니다.

#### raw_corpus 로 데이터를 더 자세히 보도록 하겠습니다! (길어서 삭제)

In [2]:
# raw_corpus
# 풀어서 실행해보세요!

훑어보면서 특수 문자나 불필요한 기호가 있어보이는지 살펴봅니다.

- '!(exclamation mark)', '?(question mark)'가 자주 보입니다.
- 문장 속 연결 부분에  ',(comma)' 도 많이 보이네요. ex) 'Oh no, oh no, no, no Okay, all right, oh no'
- '( )' 괄호 안에 들어간 것도 가사이니 괄호도 삭제해주면 좋을 것 같습니다.
- 가끔 '-(hyphen)'도 보입니다. ex) 'a-wigglin'

#### 이제 데이터 정제를 시작합니다.

- 여기서 찾아둔 필요 없는 기호 등을 삭제하고, 필요한 문장과 필요 없는 문장을 구분하여 삭제 또는 수정을 통해 데이터 정제를 시작합니다.

# 3. 데이터 정제
## 3.(1) 표준화 (Standardize)
- 표준화는 일반적으로 데이터 세트를 단순화하기 위해 구두점이나 HTML 요소를 제거하기 위해 텍스트를 전처리하는 것을 말합니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        # 길이가 0인 문장 삭제

    if idx > 9: break
        # 일단 문장 10개만 확인해보기
        
    print(sentence)

At first I was afraid
I was petrified
I kept thinking I could never live without you
By my side But then I spent so many nights
Just thinking how you've done me wrong
I grew strong
I learned how to get along And so you're back
From outer space
I just walked in to find you
Here without that look upon your face I should have changed that fucking lock


#### 아래 코드 데이터 정제 순서

1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 '<'start'>', 끝에는 '<'end'>'를 추가합니다
- 이 순서로 직접 텍스트를 표준화 해주는 함수를 작성해 봅시다! 위의 순서대로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요! 

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([-?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 문장 하나를 필터링한 예시 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

In [6]:
len(corpus)

175986

- corpus에 담은 정제된 데이터가 187088개에서 175986개로 줄었음을 알 수 있네요!

## 3.(2) 토큰화 (Tokenize)
-  문자열을 토큰으로 분할하는 것을 말합니다(예: 공백에서 분할하여 문장을 개별 단어로 분할)

## 3. (3) 벡터화 (Vectorization)
- 벡터화는 토큰을 신경망에 제공할 수 있도록 숫자로 변환하는 것을 말합니다.


### 설참📜 토큰화 진행 전에 보면 좋은 TensorFlow 자료! 
- 📌[tf.keras.preprocessing.text.Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)

- 📌[tf.keras.preprocessing.sequence.pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)
- 📌[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춤
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)   
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   71  241 ...    0    0    0]
 [   2    5   57 ...    0    0    0]
 [   2    5 1087 ...    0    0    0]
 ...
 [   2   48   16 ...    0    0    0]
 [  25    9 2859 ...  264   19    3]
 [   2    6  181 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f42dcdcf990>


In [8]:
# 텐서 데이터 조회

print(tensor[:3, :10])

[[   2   71  241    5   57  665    3    0    0    0]
 [   2    5   57 6490    3    0    0    0    0    0]
 [   2    5 1087  533    5  103   79  205  258    7]]


In [9]:
# 단어 사전 조회

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


- tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
- 마지막 토큰은 '<'end'>'가 아니라 '<'pad'>'일 가능성이 높음
- num_words를 12,000개로 맞췄기 때문에 토큰이 12,000개 보다 적은 경우에는 마지막이 pad로 끝나서 가능성이 높다고 볼 수 있음

## Source, Target 으로 구분한 Dataset 생성

In [10]:
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  71 241   5  57 665   3   0   0   0   0   0   0   0]
[ 71 241   5  57 665   3   0   0   0   0   0   0   0   0]


In [11]:
print('Shape of Tenser :', tensor.shape)
print('Shape of Source :', src_input.shape)
print('Shape of Target :', tgt_input.shape)

Shape of Tenser : (175986, 15)
Shape of Source : (175986, 14)
Shape of Target : (175986, 14)


- 모든 토큰이 텐서로 바뀐 것을 볼 수 있습니다!

# 4. 훈련/평가 데이터셋 분리

- tokenize() 함수로 데이터를 Tensor로 변환하였으니, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하겠습니다.

In [12]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=32)

In [13]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print('Source Test :', enc_val.shape)
print('Target Test :', dec_val.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)
Source Test : (35198, 14)
Target Test : (35198, 14)


In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

지금까지의 과정을 다시 한 번 정리해보면 기까지 해온 과정을 다시 한 번 정리해보면
1. 정규표현식을 이용한 corpus 생성
2. tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
3. tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

## 5. 모델로 학습하기

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.33203580e-04, -1.86599616e-04,  2.18321511e-04, ...,
         -1.53840359e-04,  9.91375855e-05, -2.03850475e-04],
        [ 2.64370115e-04, -2.77696119e-04,  4.84785269e-04, ...,
         -1.53165965e-04, -1.92066873e-04, -3.30028706e-04],
        [ 2.64633854e-04, -1.82564065e-04,  1.07968121e-03, ...,
         -1.84216202e-04, -4.24272526e-04, -1.33159425e-04],
        ...,
        [ 2.66692281e-04, -4.34181129e-04,  7.11414381e-04, ...,
          6.34632888e-04, -4.73407796e-04,  1.06723257e-03],
        [ 2.30330828e-04, -4.13973961e-04,  4.17247385e-04, ...,
          6.72830793e-04, -4.29097534e-04,  9.87319043e-04],
        [ 1.99396527e-04, -4.05011931e-04,  1.50480744e-04, ...,
          6.77563075e-04, -4.16178082e-04,  8.68083036e-04]],

       [[ 2.33203580e-04, -1.86599616e-04,  2.18321511e-04, ...,
         -1.53840359e-04,  9.91375855e-05, -2.03850475e-04],
        [ 4.69908933e-04, -2.05484961e-04,  7

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


- 현재 만든 신경망 모델은 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있습니다.
- Embedding 레이어는 정수로 인코딩된 리뷰를 가져와 각 단어 인덱스에 대한 임베딩 벡터를 찾습니다. 이러한 벡터는 모델이 학습될 때 학습됩니다. 벡터는 출력 배열에 차원을 추가합니다. 결과 치수는 다음과 같습니다 (batch, sequence, embedding)
- LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다.

In [78]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
687/687 [==============================] - 245s 352ms/step - loss: 4.1124
Epoch 2/30
687/687 [==============================] - 244s 355ms/step - loss: 3.1728
Epoch 3/30
687/687 [==============================] - 244s 355ms/step - loss: 2.9579
Epoch 4/30
687/687 [==============================] - 244s 354ms/step - loss: 2.8031
Epoch 5/30
687/687 [==============================] - 243s 353ms/step - loss: 2.6699
Epoch 6/30
687/687 [==============================] - 243s 353ms/step - loss: 2.5539
Epoch 7/30
687/687 [==============================] - 243s 353ms/step - loss: 2.4443
Epoch 8/30
687/687 [==============================] - 243s 353ms/step - loss: 2.3385
Epoch 9/30
687/687 [==============================] - 243s 353ms/step - loss: 2.2400
Epoch 10/30
687/687 [==============================] - 242s 352ms/step - loss: 2.1530
Epoch 11/30
687/687 [==============================] - 242s 352ms/step - loss: 2.0638
Epoch 12/30
687/687 [==============================] - 242s 352

- epoch 30을 실행하는 동안 loss는 4.11에서 loss 1.0172으로 내려갔습니다.

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다.
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다.
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다.
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다.
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor,
                                 tf.expand_dims(predict_word, axis=0)],
                                axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he formulate flavors darn darn cant cant cant fired fired fired fired barred barred barred barred hypnotize scented heathen '

- 'he'로 시작하는 문장을 만들었더니 'he s a monster'이란 문장이 처음 나왔다가 뒤에는 조금 희안한 문장이 나왔습니다.(아마 오래 걸려 모델 학습을 안 시켜서 그런 것 같습니다.)
- 문법적으로 조금 아쉬운 부분은 있지만, 그래도 문장이 나왔다는 사실에 되게 기쁩니다.

## 6. 프로젝트 정리

이번 Exploration 04번째 프로젝트는 노래 가사가 담긴 Dataset을 통해 작사를 해주는 인공지능을 만들어 보는 프로젝트였습니다.

- 원데이터로 약 187088 문장이 담겨 있는 데이터였습니다.

프로젝트를 통해 배울 수 있었던 것은,

1. text 데이터는 어떻게 데이터를 전처리 해야하는지?
- 표준화, 토큰화, 벡터화의 기본 기능을 익힐 수 있었습니다.
2. 훈련/평가 데이터를 분리하여 모델을 통해 학습하는 방법
-  Sequential 모델 등 다른 model을 구성해보려고 했지만, 오류로 lms에서 기본적으로 구성해줬던 모델로 학습을 할 수밖에 없었습니다.
3. epochs를 30으로 돌렸을 때 loss가 많이 낮아졌지만 10으로 했을 때는 loss가 2.2보다 높을 것으로 예상됩니다.



프로젝트 후 개인적으로 추가 공부를 해보자!
- [Guide to the Sequential model](https://keras.io/ko/getting-started/sequential-model-guide/)
- [Text classification guide](https://developers.google.com/machine-learning/guides/text-classification/)